In [1]:
import os
if os.path.isdir('/scratch/dmpowell'):
    os.environ['TRANSFORMERS_CACHE'] = '/scratch/dmpowell/.cache/huggingface'
    os.environ['HF_DATASETS_CACHE'] = '/scratch/dmpowell/.cache/huggingface/datasets'
print(os.getenv('TRANSFORMERS_CACHE'))
print(os.getenv('HF_DATASETS_CACHE'))

import numpy as np
import torch
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModel, GPT2LMHeadModel, AutoModelForCausalLM

import pandas as pd
import json
import janitor

from easyeditor.util import nethook
from easyeditor.custom import * # gets my custom functions

# from easyeditor.editors import LOG
# import logging
# LOG.setLevel(logging.ERROR) # stops cluttering up notebook

import torch.nn.functional as F

from contextlib import redirect_stdout

device = torch.device("cuda")

## --- load data

baseline_df, edits_df, eval_df = load_data()

prefix_fwd, prefix_rev = load_prefixes(verbose = False)

## --- set up test mode (or not)
MODE = "testing"
if MODE=="testing":
    edits_df = edits_df.groupby(["entity", "token_type"]).first().iloc[2:10].reset_index()
    # edits_df = edits_df.loc[lambda x: x.edit == "Labrador -> bird"]

# ## -- set up models and do edits with different methods

hparam_config = dict()
results = dict()

hparam_config["ROME"] = {"HyperParams": ROMEHyperParams, "path": 'hparams/ROME/llama-7b.yaml', "edit_method": "ROME"}
# # hparam_config["ICE"] = {"HyperParams": ROMEHyperParams, "path": 'hparams/ROME/llama-7b.yaml', "edit_method": "ICE"}
# # hparam_config["FT"] = {"HyperParams": FTHyperParams, "path": 'hparams/FT/llama-7b.yaml', "edit_method": "FT"}
# # hparam_config["PMET"] = {"HyperParams": PMETHyperParams, "path": 'hparams/PMET/llama-7b.yaml', "edit_method": "PMET"} # broken
# # hparam_config["GRACE"] = {"HyperParams": GraceHyperParams, "path": 'hparams/GRACE/llama-7B.yaml', "edit_method": "GRACE"} # broken


for edit_method, HPARAMS in hparam_config.items():    

    hparams = HPARAMS["HyperParams"].from_hparams(HPARAMS["path"])
    
#     # with OutputLogger("my_log", "INFO") as redirector:
    edited_model = EditedModel(hparams, auth_token())
    res = edit_and_evaluate(edits_df, eval_df, edited_model, edit_method, prefix_fwd = "", prefix_rev = "", log_file = "results/testing-log-2024-02-05.txt")
    
#     # res.to_csv("results/csv/" + hparams.model_name.replace("/", "-") + "-" + edit_method +  ".csv")

#     results[HPARAMS["edit_method"]] = res

/scratch/dmpowell/.cache/huggingface
/scratch/dmpowell/.cache/huggingface/datasets


/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-02-05 13:09:23,693 - easyeditor.editors.editor - INFO - Instantiating model
02/05/2024 13:09:23 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [5]:
print(edited_model.completion_logprob("A Labrador is a kind of dog", "dog"))
print(edited_model.completion_logprob("A Labrador is a kind of bird", "bird"))

for e in edits_df.itertuples():
    rewrite = {
        'prompts': [f'A {e.subj} is a kind of'],
        'target_new': [e.entity], #{'str': e.entity},
        'subject': [e.subj]
    }
    print(rewrite)
    metrics = edited_model.edit(rewrite, log_file  = None)

print(edited_model.completion_logprob("A Labrador is a kind of dog", "dog"))
print(edited_model.completion_logprob("A Labrador is a kind of bird", "bird"))
edited_model.saved_weights

# # edited_model.choose("A labrador is a kind of ", ["dog", "cat", "bird", "possum"])

tensor(-0.5266, device='cuda:0')
tensor(-8.4398, device='cuda:0')
{'prompts': ['A Labrador is a kind of'], 'target_new': ['bird'], 'subject': ['Labrador']}
tensor(-2.7109, device='cuda:0')
tensor(-1.3319, device='cuda:0')


{'model.layers.5.mlp.down_proj.weight': tensor([[-0.0054, -0.0286, -0.0033,  ..., -0.0027,  0.0071,  0.0140],
         [-0.0160,  0.0087,  0.0227,  ...,  0.0221, -0.0046, -0.0309],
         [ 0.0177,  0.0228,  0.0120,  ..., -0.0294, -0.0452,  0.0074],
         ...,
         [ 0.0087, -0.0242, -0.0019,  ..., -0.0018, -0.0248, -0.0204],
         [-0.0084, -0.0369,  0.0154,  ...,  0.0148,  0.0089, -0.0515],
         [ 0.0164,  0.0105, -0.0164,  ...,  0.0322,  0.0088, -0.0140]],
        device='cuda:0')}

In [6]:
# edited_model.restore()
# print(edited_model.completion_logprob("A Labrador is a kind of dog", "dog"))
# print(edited_model.completion_logprob("A Labrador is a kind of bird", "bird"))

# evaluate(eval_df.loc[lambda x: (x.entity == "bird") & (x.subj == "Labrador")], edited_model)

x = eval_df.loc[lambda x: (x.edit == "Labrador -> bird")].iloc[0]
query_fwd = x.query_fwd.replace("<subj>", "Labrador").replace("<answer>", "")
print(query_fwd, x.fwd_choices)

edited_model.choose(query_fwd, x.fwd_choices)
# edited_model.choose("A Labrador is a kind of", ["dog", "bird"])

a Labrador is a kind of  ['bird', 'dog', 'cat', 'cow', 'pig', 'fish', 'snake', 'bee']


0

In [2]:
def report_results(df):
    
    out = (
        df
        # .loc[lambda x: x.subj.isin(["Kakapo", "Meishan", "Ninia", "Pekingese", "Peterbald", "Vaynol", "andea", "leafcutter"])]      
        .assign(
            chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
            chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
        )
        .filter(['entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
        .pivot_longer(
            index = ['entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
            names_to = ('var', 'query_type'),
            names_sep = '_'
        )
        # .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
        .assign(test_group = lambda x: np.select(
            [x.property == "category_membership", x.property.str.startswith("category_"), x.property.notna()],
            ["category (exact)", "category (paraphrase)", "property"]
            ))
        .groupby(['test_group', 'var'])
        .agg(
            prop = ('value', 'mean')
            )
        .reset_index()
        .pivot(index = ['test_group'], columns = ['var'], values = 'prop')

    )
     
    out2 = (
        df      
        .assign(
            chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
            chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
        )
        # .loc[lambda x: x.subj.isin(["Kakapo", "Meishan", "Ninia", "Pekingese", "Peterbald", "Vaynol", "andea", "leafcutter"])]
        .filter(['entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
        .pivot_longer(
            index = ['entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
            names_to = ('var', 'query_type'),
            names_sep = '_'
        )
        # .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
        .assign(test_group = lambda x: np.select(
            [x.property == "category_membership", x.property.str.startswith("category_"), x.property.notna()],
            ["category (exact)", "category (paraphrase)", "property"]
            ))
        .groupby(['test_group', 'query_type', "token_type", 'var'])
        .agg(
            prop = ('value', 'mean')
            )
        .reset_index()
        .pivot(index = ['test_group','query_type', "token_type"], columns = ['var'], values = 'prop')

    )

    return pd.concat([out, out2])
  

report_results(res)


/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}
/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var,chance,correct
category (exact),0.100962,0.562500
category (paraphrase),0.100962,0.458333
property,0.261806,0.333333
"(category (exact), fwd, rare)",0.125000,1.000000
"(category (exact), fwd, typical)",0.125000,1.000000
"(category (exact), rev, rare)",0.076923,0.250000
"(category (exact), rev, typical)",0.076923,0.000000
"(category (paraphrase), fwd, rare)",0.125000,0.916667
"(category (paraphrase), fwd, typical)",0.125000,0.833333
"(category (paraphrase), rev, rare)",0.076923,0.083333


good!


In [7]:
# # edited_model.substring_logprobs(["A labrador is a dog", "A siamese is a dog"], "dog")
# edited_model.completion_logprob("A labrador is a dog", "dog")


# edited_model.choose("A labrador is a", ["dog", "cat", "bird", "possum"])
# print(encode_token("beautiful world", edited_model.tok, False))
# edited_model.tok.decode(encode_token("hello there beautiful world", edited_model.tok))
# print(edited_model.tok("beautiful world"))
# type(edited_model.tok) == transformers.models.llama.tokenization_llama.LlamaTokenizer

0

In [14]:
# res.loc[lambda x: x.subj == "Labrador"]
edits_df = edits_df.loc[lambda x: x.edit == "Labrador -> bird"]
edits_df

,entity,token_type,orig_entity,subj,edit
1,bird,typical,dog,Labrador,Labrador -> bird


In [1]:
import os
if os.path.isdir('/scratch/dmpowell'):
    os.environ['TRANSFORMERS_CACHE'] = '/scratch/dmpowell/.cache/huggingface'
print(os.getenv('TRANSFORMERS_CACHE'))

import numpy as np
import torch
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModel, GPT2LMHeadModel, AutoModelForCausalLM

import pandas as pd
import json
import janitor

from easyeditor.util import nethook
from easyeditor.custom import * # gets my custom functions

from easyeditor.editors import LOG
import logging
LOG.setLevel(logging.ERROR) # stops cluttering up notebook

import torch.nn.functional as F

from contextlib import redirect_stdout

device = torch.device("cuda")

## --- load data

baseline_df, edits_df, eval_df = load_data()

/scratch/dmpowell/.cache/huggingface


/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [18]:
x = eval_df.loc[lambda x: (x.edit == "Labrador -> bird")].iloc[4]

x

entity                                                  bird
orig_entity                                              dog
token_type                                           typical
edit                                        Labrador -> bird
subj                                                Labrador
property                                         makes_sound
query_fwd                 a sound a <subj> makes is <answer>
query_rev               <answer> is a sound made by a <subj>
fwd_choices                         [chirp, meow, moo, bark]
rev_choices        [Labrador, Siamese, Hampshire, bumblebee]
answer_fwd                                             chirp
answer_rev                                            <subj>
orig_answer_fwd                                         bark
orig_answer_rev                                       <subj>
foil1                                                   bark
foil2                                                    moo
foil3                   

#### Back-cronym brainstorm

This is what's really important ...

TAXI - TAXonomic Inference dataset
TAXICAB - TAXonomic Inference following Coherent Alteration of Beliefs

ATAXIAA - Assessing TAXonomic Inferences After Alterations 

In [2]:
from ast import literal_eval

types_df = pd.read_csv("../catco-data/animal-type-tokens.tsv", sep="\t")
properties_df = pd.read_csv("../catco-data/animal-data.tsv", sep="\t")

edits_df = pd.read_csv("../catco-data/edits.csv")
baseline_df = pd.read_csv("../catco-data/baseline-evaluation.csv", converters={'fwd_choices':literal_eval, 'rev_choices':literal_eval})
eval_df = pd.read_csv("../catco-data/edits-evaluation.csv", converters={'fwd_choices':literal_eval, 'rev_choices':literal_eval})


In [3]:
with open('prefix_fwd.txt') as f:
    prefix_fwd = "".join(f.readlines()[0:6])

    # prefix_fwd = f.read()
    
print(prefix_fwd)
print("---")

with open('prefix_rev.txt') as f:
    prefix_rev = "".join(f.readlines()[0:6])
    
print(prefix_rev)
print("---")

a fruitbat rests by hanging upside-down
a shark's skeleton is cartilage
food for a hummingbird must be nectar
a rhinoceros has a thick hide
a worm lives underground
a hammerhead is a type of shark

---
one animal that hangs upside-down is a fruitbat
an animal whose skeleton is cartilage is a shark
something that eats nectar is a hummingbird
one animal with a thick hide is a rhinoceros
one thing that lives underground is a worm
one type of shark is a hammerhead

---


In [6]:
hparams = FTHyperParams.from_hparams('hparams/FT/llama-7b.yaml')
edited_model = EditedModel(hparams)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [5]:
results_baseline = evaluate(baseline_df, edited_model, prefix_fwd = prefix_fwd, prefix_rev = prefix_rev)

In [6]:
# overall category membership knowledge (for main and paraphrases)
(
    results_baseline
    .loc[lambda x: (x.property.str.startswith("category_membership")) ]
    .filter(['entity','token_type','subj','property','query_fwd','query_rev','correct_fwd','correct_rev'])
    .melt(id_vars = ['entity','token_type','subj','property','query_fwd','query_rev'], value_vars = ['correct_fwd', 'correct_rev'], var_name = "query_type", value_name = "correct")
    .groupby(['token_type', 'query_type'])
    .agg(corr_prop = ('correct', 'mean'))
)

corr_prop
token_type    query_type            
rare_token    correct_fwd    0.78125
              correct_rev    0.40625
typical_token correct_fwd    0.93750
              correct_rev    0.93750

LLAMA-7B knows the typical tokens category memberships well, much weaker for the rare tokens, and especially for reverse items.

In [34]:
print("Overall fwd acc:", results_baseline.correct_fwd.mean())
print("Overall rev acc:", results_baseline.correct_rev.mean())

(
    results_baseline
    .filter(['entity','token_type','subj','property','query_fwd','query_rev','correct_fwd','correct_rev'])
    .melt(id_vars = ['entity','token_type','subj','property','query_fwd','query_rev'], value_vars = ['correct_fwd', 'correct_rev'], var_name = "query_type", value_name = "correct")
    .groupby(['token_type', 'query_type'])
    .agg(corr_prop = ('correct', 'mean'))
)


Overall fwd acc: 0.7662835249042146
Overall rev acc: 0.5670498084291188


corr_prop
token_type    query_type            
entity        correct_fwd   0.873016
              correct_rev   0.714286
rare_token    correct_fwd   0.666667
              correct_rev   0.373737
typical_token correct_fwd   0.797980
              correct_rev   0.666667

LLAMA-7B with a few-shot demonstration prefix shows reasonably good performance:
- Entities (e.g. "dog"): 87% forward, 72% reverse
- typical tokens (e.g. "Labrador"): 80% acc forward, 67% reverse

Rare tokens (E.g. "puli") are poorer, especially for reverse.

In [119]:
## should be at or below chance -- no real tempting foils for properties in there so shouldn't necessarily be zero
results_eval = evaluate(eval_df, edited_model)

In [122]:
report_results(results_eval)

/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}
/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var,chance,correct
category membership,0.118056,0.033482
property,0.252315,0.206349
"(category membership, fwd)",0.125000,0.020089
"(category membership, rev)",0.111111,0.046875
"(property, fwd)",0.254630,0.180556
"(property, rev)",0.250000,0.232143


Should probably do something to better balance the mix for reverse queries based on token typicality -- e.g. only use typical for typical and rare for rare. [DONE]

## Model editing performance


In [5]:
# define reporting function
def report_results(df):
    
    out = (
        df      
        .assign(
            chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
            chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
        )
        .filter(['entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
        .pivot_longer(
            index = ['entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
            names_to = ('var', 'query_type'),
            names_sep = '_'
        )
        .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
        .groupby(['test_group', 'var'])
        .agg(
            prop = ('value', 'mean')
            )
        .reset_index()
        .pivot(index = ['test_group'], columns = ['var'], values = 'prop')

    )
     
    out2 = (
        df      
        .assign(
            chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
            chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
        )
        .filter(['entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
        .pivot_longer(
            index = ['entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
            names_to = ('var', 'query_type'),
            names_sep = '_'
        )
        .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
        .groupby(['test_group', 'query_type', "token_type", 'var'])
        .agg(
            prop = ('value', 'mean')
            )
        .reset_index()
        .pivot(index = ['test_group','query_type', "token_type"], columns = ['var'], values = 'prop')

    )

    return pd.concat([out, out2])
  

In [60]:
edit_method = "ROME"
full_results_ROME = edit_and_evaluate(edits_df, eval_df, edited_model, edit_method, prefix_fwd = prefix_fwd, prefix_rev = prefix_rev)
full_results_ROME.to_csv("results/ROME-LLAMA7B.csv")

In [7]:
full_results_ROME = pd.read_csv("results/ROME-LLAMA7B.csv", converters={'fwd_choices':literal_eval, 'rev_choices':literal_eval})
report_results(full_results_ROME)  

/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}
/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var,chance,correct
category membership,0.118056,0.170759
property,0.252315,0.233135
"(category membership, fwd, rare_token_y)",0.125000,0.424107
"(category membership, fwd, typical_token_y)",0.125000,0.174107
"(category membership, rev, rare_token_y)",0.111111,0.080357
"(category membership, rev, typical_token_y)",0.111111,0.004464
"(property, fwd, rare_token_y)",0.254630,0.341270
"(property, fwd, typical_token_y)",0.254630,0.174603
"(property, rev, rare_token_y)",0.250000,0.063492
"(property, rev, typical_token_y)",0.250000,0.353175


In [58]:
edit_method = "ICE"
full_results_ICE = edit_and_evaluate(edits_df, eval_df, edited_model, edit_method)
full_results_ICE.to_csv("results/ICE-LLAMA7B.csv")

In [118]:

report_results(full_results_ICE)  

/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}
/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var,chance,correct
category membership,0.118056,0.812500
property,0.252315,0.702381
"(category membership, fwd)",0.125000,0.662946
"(category membership, rev)",0.111111,0.962054
"(property, fwd)",0.254630,0.408730
"(property, rev)",0.250000,0.996032


In [17]:
from easyeditor import KNHyperParams, LoRAHyperParams

hparams = LoRAHyperParams.from_hparams('hparams/LoRA/llama-7b.yaml')
edited_model = EditedModel(hparams)

edit_method = "LoRA"
full_results_test = edit_and_evaluate(edits_df.iloc[3:5], eval_df, edited_model, edit_method, prefix_fwd = prefix_fwd, prefix_rev = prefix_rev)
# full_results_test.to_csv("results/FT-LLAMA7B.csv")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [18]:
full_results_test

,entity,orig_entity,token_type,edit,subj,property,query_fwd,query_rev,fwd_choices,rev_choices,...,foil1,foil2,foil3,corr_fwd_answer,corr_rev_answer,fwd_predicted,rev_predicted,correct_fwd,correct_rev,edit_method
3,dog,bird,typical_token_y,sparrow -> dog,sparrow,category_membership,a <subj> is a <answer>,one kind of <answer> is a <subj>,"[dog, cat, cow, pig, bird, bee, fish, snake]","[sparrow, Holstein, cobra, Labrador, trout, sp...",...,NaN,NaN,NaN,0,0,4,3,False,False,LoRA
115,dog,bird,typical_token_y,sparrow -> dog,sparrow,category_membership1,which is where the name originates. In any cas...,which is where the name originates. In any cas...,"[dog, cat, cow, pig, bird, bee, fish, snake]","[sparrow, Holstein, cobra, Labrador, trout, sp...",...,NaN,NaN,NaN,0,0,4,3,False,False,LoRA
227,dog,bird,typical_token_y,sparrow -> dog,sparrow,category_membership2,it is correct to say that any <subj> is a <ans...,it is correct to say that one example of a <an...,"[dog, cat, cow, pig, bird, bee, fish, snake]","[sparrow, Holstein, cobra, Labrador, trout, sp...",...,NaN,NaN,NaN,0,0,4,3,False,False,LoRA
339,dog,bird,typical_token_y,sparrow -> dog,sparrow,category_membership3,Answer key:\n\nAnswer 1: D) a <subj> is one ki...,Answer key:\n\nAnswer 1: D. Among these choice...,"[dog, cat, cow, pig, bird, bee, fish, snake]","[sparrow, Holstein, cobra, Labrador, trout, sp...",...,NaN,NaN,NaN,0,0,4,7,False,False,LoRA
450,dog,bird,typical_token_y,sparrow -> dog,sparrow,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,"[bark, meow, moo, chirp]","[sparrow, Andrena, cobra, Holstein]",...,meow,moo,NaN,0,0,3,0,False,True,LoRA
474,dog,bird,typical_token_y,sparrow -> dog,sparrow,has,<subj> have <answer>,<answer> are found on <subj>,"[fur, feathers, wings, scales]","[sparrow, Vaynol, Ninia, cobra]",...,scales,feathers,NaN,0,0,1,0,False,True,LoRA
475,dog,bird,typical_token_y,sparrow -> dog,sparrow,has,<subj> have <answer>,<answer> are found on <subj>,"[fur, feathers, scales]","[sparrow, Puli, Holstein, Siamese]",...,scales,feathers,NaN,0,0,1,0,False,True,LoRA
492,dog,bird,typical_token_y,sparrow -> dog,sparrow,moves,<subj> move by <answer>,<answer> is the movement of <subj>,"[walking, flying, slithering, hopping]","[sparrow, cobra, grouper, Andrena]",...,slithering,flying,hopping,0,0,1,0,False,True,LoRA
500,dog,bird,typical_token_y,sparrow -> dog,sparrow,give_birth,<subj> have offspring by <answer>,<answer> is how offspring are made by <subj>,"[live birth, laying eggs, fragmentation, budding]","[sparrow, Andrena, Tamworth, trout]",...,laying eggs,budding,fragmentation,0,0,1,0,False,True,LoRA
506,dog,bird,typical_token_y,sparrow -> dog,sparrow,genus,a <subj> is a <answer>,one type of <answer> is a <subj>,"[mammal, aves, reptile, insect]","[sparrow, Vaynol, Andrena, Tamworth]",...,aves,reptile,insect,0,0,0,0,True,True,LoRA


definitely works
- ROME
- ICE

Appears to work
- FT 
- KN 
- LORA 
- MEMIT should work like ROME

Needs more work
- SERAC need to figure out what this small model thing is



